# ALIKED demo resultados

In [ ]:
#import torch
#print(torch.cuda.is_available())

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Clonar repositorio
!pip install opencv-python kornia torch torchvision --quiet
!git clone https://github.com/Shiaoming/ALIKED.git
%cd ALIKED

In [ ]:
# 2. Instalar dependencias

#!pip install -r requirements.txt

In [ ]:
# 3. Compilar  operaciones personalizadas

#%cd /content/ALIKED/custom_ops
#!python setup.py build_ext --inplace
#!sh build.sh
#%cd /content/ALIKED

In [ ]:
#from custom_ops import get_patches
#print("Modulo compilado e importado correctamente.")


In [ ]:
# 2. Importar librerías y cargar modelo
import os
import cv2
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.stats import entropy

from kornia.feature import ALIKED

device = torch.device("cpu")
model = ALIKED(pretrained='outdoor', max_num_keypoints=1000).to(device).eval()  # usamos 1000, pero luego filtramos Top-N(solo prueba)


In [ ]:
# 3. Funciones auxiliares
def extract_aliked_features(image, top_n=500):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_tensor = torch.from_numpy(gray / 255.).float().unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(img_tensor)

    keypoints = out.keypoints[0].cpu().numpy()
    scores = out.scores[0].cpu().numpy()
    descriptors = out.descriptors[0].cpu().numpy()

    # Top-N keypoints por score
    if len(scores) > top_n:
        idx = np.argsort(scores)[::-1][:top_n]
        keypoints = keypoints[idx]
        descriptors = descriptors[idx]

    return keypoints, descriptors

def match_features(desc1, desc2):
    distances = cdist(desc1, desc2, metric='euclidean')
    indices = np.argmin(distances, axis=1)
    min_distances = distances[np.arange(distances.shape[0]), indices]
    matches = np.stack((np.arange(len(indices)), indices), axis=1)
    return matches, min_distances

def spatial_entropy(keypoints, image_shape, grid_size=4):
    H, W = image_shape[:2]
    heatmap = np.zeros((grid_size, grid_size))
    for kp in keypoints:
        x, y = kp[:2]
        col = min(int(x / W * grid_size), grid_size - 1)
        row = min(int(y / H * grid_size), grid_size - 1)
        heatmap[row, col] += 1
    prob = heatmap.flatten()
    prob = prob / (prob.sum() + 1e-8)
    return entropy(prob)

def draw_matches(img1, kp1, img2, kp2, matches, save_path):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    new_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    new_img[:h1, :w1] = img1
    new_img[:h2, w1:] = img2
    for match in matches:
        pt1 = tuple(np.round(kp1[match[0], :2]).astype(int))
        pt2 = tuple(np.round(kp2[match[1], :2]).astype(int) + np.array([w1, 0]))
        cv2.line(new_img, pt1, pt2, (0, 255, 0), 1)
    cv2.imwrite(save_path, new_img)


In [ ]:
# 4. Procesamiento por pares
# Ruta base en Google Drive
base_path = '/content/drive/MyDrive/pruebas-pfc-2025/aachen_dataset/day'
result_dir = '/content/aliked_day_results'
os.makedirs(result_dir, exist_ok=True)

csv_data = []

for folder in tqdm(sorted(os.listdir(base_path))):
    pair_path = os.path.join(base_path, folder)
    img1_path = os.path.join(pair_path, 'img1.png')
    img2_path = os.path.join(pair_path, 'img2.png')

    if os.path.exists(img1_path) and os.path.exists(img2_path):
        try:
            img1 = cv2.imread(img1_path)
            img2 = cv2.imread(img2_path)

            kp1, desc1 = extract_aliked_features(img1, top_n=500)
            kp2, desc2 = extract_aliked_features(img2, top_n=500)

            matches, distances = match_features(desc1, desc2)

            num_matches = len(matches)
            efficiency = num_matches / (len(desc1) + 1e-8)
            avg_distance = float(np.mean(distances))

            matched_kp1 = np.array([kp1[m[0]] for m in matches])
            spatial_ent = spatial_entropy(matched_kp1, img1.shape)

            output_img_path = os.path.join(result_dir, f'{folder}_matches.png')
            draw_matches(img1, kp1, img2, kp2, matches, output_img_path)

            csv_data.append({
                'pair': folder,
                'num_matches': num_matches,
                'efficiency': round(efficiency, 4),
                'avg_distance': round(avg_distance, 4),
                'spatial_entropy': round(spatial_ent, 4)
            })
        except Exception as e:
            print(f"Error en {folder}: {e}")

# Guardar CSV
df = pd.DataFrame(csv_data)
csv_path = os.path.join(result_dir, 'aliked_day_results.csv')
df.to_csv(csv_path, index=False)
print(f"Resultados guardados en: {csv_path}")
